# <span style="color:#bce35b;font-size:35px"><b>Week 7 & 8</b></span>
<!-- # <span style="color:#bce35b;font-size:35px">Title<b>Title</b></span> -->
<!-- # <span style="color:#bce35b">H1</span> -->
<!-- ## <span style="color:#ae8bd5">H2</span> -->
<!-- ### <span style="color:#9c4957">H3</span> -->
<!-- #### <span style="color:white">H4</span> -->

<span style="color:#8c8c8c">Patrick Weatherford</span>

[Green]: <> (#bce35b)
[Purple]: <> (#ae8bd5)
[Coral]: <> (#9c4957)
[Grey]: <> (#8c8c8c)

***
***

Importing libraries

In [1]:
import os
import keyring

os.environ['KAGGLE_USERNAME'] = 'hakunapatata'
os.environ['KAGGLE_KEY'] = keyring.get_password(service_name='kaggle_api', username=os.environ['KAGGLE_USERNAME'])

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import requests
from collections import deque as deq
from itertools import permutations, dropwhile, zip_longest
import re
import math
import io
import scipy
import sys
from bs4 import BeautifulSoup
import lxml
from fuzzywuzzy import fuzz, process
import sqlite3
import datetime as dt
from dask import dataframe as dd
from contextlib import closing

%matplotlib inline

## matplotlib default settings
plt.style.use('dark_background')
mpl.rcParams.update({'lines.linewidth':3})
mpl.rcParams.update({'axes.labelsize':14})
mpl.rcParams.update({'axes.titlesize':16})
mpl.rcParams.update({'axes.titleweight':'bold'})
mpl.rcParams.update({'figure.autolayout':True})
mpl.rcParams.update({'axes.grid':True, 'grid.color':'#424242', 'grid.linestyle':'--'})

## creation random number generator object
rng_seed = 777
rng = np.random.default_rng(rng_seed)


In [2]:
## see list of current settings
# plt.rcParams.keys()

<br>

***

# <span style="color:#bce35b">Week Activities</span>
<!-- # <span style="color:#bce35b;font-size:35px">Title<b>Title</b></span> -->
<!-- # <span style="color:#bce35b">H1</span> -->
<!-- ## <span style="color:#ae8bd5">H2</span> -->
<!-- ### <span style="color:#9c4957">H3</span> -->
<!-- #### <span style="color:white">H4</span> -->

<!-- <span style="color:#8c8c8c">Patrick Weatherford</span> -->

[Green]: <> (#bce35b)
[Purple]: <> (#ae8bd5)
[Coral]: <> (#9c4957)
[Grey]: <> (#8c8c8c)

From the datasets provided, manipulate/clean the dataset using 2 concepts from each of the chapters below.

<span style="font-size:12px">

Chapter 7
- Filter out missing data
- Fill in missing data
- Remove duplicates
- Transform data using either mapping or a function
- Replace values
- Discretization and Binning
- Manipulate Strings

Chapter 8
- Create hierarchical index
- Combine and Merge Datasets (you will have to either create a new dataset from your existing data or create a relationship between the data I have provided)
- Reshape
- Pivot the data

Chapter 10
- Grouping with Dicts/Series
- Grouping with Functions
- Grouping with Index Levels
- Split/Apply/Combine
- Cross Tabs

Chapter 11
- Convert between string and date time
- Generate date range
- Frequencies and date offsets
- Convert timestamps to periods and back
- Period Frequency conversions

</span>

Import Metropolitan Museum of Art .csv file

In [3]:
csv_url = "https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv"

bad_line_list = []

def bad_line_func(line):
    bad_line_list.append(line)

df = dd.read_csv(csv_url, dtype="string", on_bad_lines=bad_line_func, engine="python")


Object Number              string
Is Highlight               string
Is Timeline Work           string
Is Public Domain           string
Object ID                  string
Gallery Number             string
Department                 string
AccessionYear              string
Object Name                string
Title                      string
Culture                    string
Period                     string
Dynasty                    string
Reign                      string
Portfolio                  string
Constituent ID             string
Artist Role                string
Artist Prefix              string
Artist Display Name        string
Artist Display Bio         string
Artist Suffix              string
Artist Alpha Sort          string
Artist Nationality         string
Artist Begin Date          string
Artist End Date            string
Artist Gender              string
Artist ULAN URL            string
Artist Wikidata URL        string
Object Date                string
Object Begin D

In [23]:
table_name = "MET_MUSEUM"

with sqlite3.connect('HakunaPatata.db') as conn:
    with closing(conn.cursor()) as c:
        c.execute(f"DROP TABLE IF EXISTS {table_name}")
        c.execute(f"""CREATE TABLE {table_name} (
            OBJECT_NUMBER text
            , IS_HIGHLIGHT text
            , IS_TIMELINE_WORK text
            , IS_PUBLIC_DOMAIN text
            , OBJECT_ID text
            , GALLERY_NUMBER text
            , DEPARTMENT text
            , ACCESSION_YEAR text
            , OBJECT_NAME text
            , TITLE text
            , CULTURE text
            , PERIOD text
            , DYNASTY text
            , REIGN text
            , PORTFOLIO text
            , CONSTITUENT_ID text
            , ARTIST_ROLE text
            , ARTIST_PREFIX text
            , ARTIST_DISPLAY_NAME text
            , ARTIST_DISPLAY_BIO text
            , ARTIST_SUFFIX text
            , ARTIST_ALPHA_SORT text
            , ARTIST_NATIONALITY text
            , ARTIST_BEGIN_DATE text
            , ARTIST_END_DATE text
            , ARTIST_GENDER text
            , ARTIST_ULAN_URL text
            , ARTIST_WIKIDATA_URL text
            , OBJECT_DATE text
            , OBJECT_BEGIN_DATE text
            , OBJECT_END_DATE text
            , MEDIUM text
            , DIMENSIONS text
            , CREDIT_LINE text
            , GEOGRAPHY_TYPE text
            , CITY text
            , STATE text
            , COUNTY text
            , COUNTRY text
            , REGION text
            , SUBREGION text
            , LOCALE text
            , LOCUS text
            , EXCAVATION text
            , RIVER text
            , CLASSIFICATION text
            , RIGHTS_AND_REPRODUCTION text
            , LINK_RESOURCE text
            , OBJECT_WIKIDATA_URL text
            , METADATA_DATE text
            , REPOSITORY text
            , TAGS text
            , TAGS_AAT_URL text
            , TAGS_WIKIDATA_URL
            )"""
            )

        df.to_sql('MET_MUSEUM', "sqlite:///HakunaPatata.db", index=False,
                if_exists='replace', parallel=True, chunksize=1000)



In [30]:
with sqlite3.connect('HakunaPatata.db') as conn:
    with closing(conn.cursor()) as c:
        select = c.execute("SELECT * FROM MET_MUSEUM LIMIT 10").fetchall()

for row in select:
    print(row) 

('12.232.8a', 'False', 'False', 'True', '248706', '162', 'Greek and Roman Art', '1912', 'Relief of a satyr and a maenad', 'Terracotta plaque', 'Roman', 'Augustan or Julio-Claudian', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '27 B.C.–A.D. 68', '-27', '-27', 'Terracotta', '20 × 17 1/2 × 1 3/4 in., 17 lb. (50.8 × 44.5 × 4.4 cm, 7.7 kg)', 'Rogers Fund, 1912', None, None, None, None, None, None, None, None, None, None, None, 'Terracottas', None, 'http://www.metmuseum.org/art/collection/search/248706', None, None, 'Metropolitan Museum of Art, New York, NY', None, None, None)
('12.232.8b', 'False', 'False', 'True', '248707', '162', 'Greek and Roman Art', '1912', 'Relief of a satyr and a maenad', 'Terracotta plaque', 'Roman', 'Augustan or Julio-Claudian', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, '27 B.C.–A.D. 68', '-27', '68', 'Terracotta', '17 3/4 × 19 7/16 × 1 3/4 in., 14 lb. (45.1 × 